### NLP CARES - Create corpus


---

**Librerías**

In [4]:
%load_ext watermark

In [5]:
%watermark --iversions

numpy  1.17.2
re     2.2.1
spacy  2.2.3
pandas 0.25.1



In [2]:
import pandas as pd
import numpy as np
import re

#Stop words and the vocabulary in spanish
from spacy.lang.es.stop_words import STOP_WORDS
import spacy
#The vocabulary in spanish
nlp = spacy.load('es_core_news_sm')

In [ ]:
nlp.vocab["no"].is_stop = False
nlp.vocab["realizado"].is_stop = False
nlp.vocab["sin"].is_stop = False
nlp.vocab["tener"].is_stop = False
nlp.vocab["manifestó"].is_stop = False
nlp.vocab["existe"].is_stop = False
nlp.vocab["considera"].is_stop = False
nlp.vocab["estados"].is_stop = False

---

**Functions**

**Cleaning Dataset**

Create a function to cleaning diferents expresion

In [11]:
def multiple_replace(text):
    dic = {r"cia a" : "ciaa","cia m" : "ciam","sis a" : "sisa", "sis m" : "sism",
           'á' : 'a', 'é' : 'e', 'í' : 'i', 'ó' : 'o', 'ú' : 'u' }
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dic.keys())))
    # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: dic[mo.string[mo.start():mo.end()]], text.lower())

In [12]:
def clean_text(text):
    # Create a regular expression  from the dictionary keys
    if isinstance(text, str):
        dic = { r'[^\w.]' : ' ', '[ 0-9 ]' : ''}
        regex = re.compile(r'(%s)' % "|".join(dic.keys()))
        lst = regex.sub(lambda mo: dic[[ k for k in dic if re.search(k, mo.string[mo.start():mo.end()])][0]], text).lower()
    else:
        lst = ' '
    return [el for el in multiple_replace(lst).split('.') if re.search("[a-z]", el)]# split texts into individual sentences

**Tokenize and lemmatize**

Definition the function to tokenize and lemmatize (tokenizar y lematizar) the text

In [15]:
def tokenize_and_lemm_spacy(text):
    doc = nlp(text)
    lemmas = []
    for token in doc:
        if token.is_stop== False: # aprovechamos para eliminar ya las stopwords
            if token.is_alpha== True: # Nos quedamos solo con los tokens que contienen letras 
                if token.pos_ not in ['CONJ', 'ADP', 'DET']: # eliminamos nombres propios, conjunciones, determinantes
                    lemmas.append(token.lemma_.lower())
    return lemmas

def tokenize_only_spacy(text):
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_stop== False :
            if token.is_alpha== True:
                if token.pos_ not in ['CONJ', 'ADP', 'DET']:
                    tokens.append(token.text.lower())
    return tokens

In [16]:
def tokenize_and_lemm_spacy_lst(lst):
    return [tokenize_and_lemm_spacy(el) for el in lst]

def tokenize_only_spacy_lst(lst):
    return [tokenize_only_spacy(el) for el in lst]

---
**Import Dataset** CAMBIAR A LEER HDF5

In [17]:
df_train = pd.read_hdf('../../data/corpus_train.h5', 'corpus')

---
**Cleaning Dataset**


Cleaning the columns "CONCLUSION" and "REASONFORSTUDY" like his name with "*\_CLEAN" for not modificate the original column

In [34]:
%%time 

df_train['document_sentences'] = df_train['CONCLUSION'].apply(clean_text)

df_train['tokenized_sentences'] = df_train['document_sentences'].apply(tokenize_and_lemm_spacy_lst)

CPU times: user 43min 38s, sys: 2.96 s, total: 43min 41s
Wall time: 44min 13s


---
**TRAIN dataset**

TRAIN: Create a new dataset with all data

In [18]:
df_train.shape

(26969, 8)

---
**Export TRAIN**

In [36]:
df_train.to_hdf('../../data/data.h5', key='TRAIN', mode='w')

---
---
## Mediconnect Dataset

**TEST dataset**

TEST: Create a new dataset with mediconet data

In [25]:
df_test = pd.read_hdf('../../data/corpus_test.h5', 'corpus')

---
**Cleaning Dataset**

Cleaning the columns "CONCLUSION" like his name with "*\_CLEAN" for not modificate the original column

In [56]:
%%time 

df_test['document_sentences'] = df_test['CONCLUSION'].apply(clean_text)

df_test['tokenized_sentences'] = df_test['document_sentences'].apply(tokenize_and_lemm_spacy_lst)

CPU times: user 5min 1s, sys: 472 ms, total: 5min 2s
Wall time: 5min 3s


---
**Export TEST**

In [31]:
df_test.to_hdf('../../data/test.h5', key='TEST', mode='w')

/home/ubuntu/anaconda3/envs/NLP/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['CONCLUSION', 'document_sentences', 'tokenized_sentences']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)
